In [1]:
text = '''
Głównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w Mainz. Akcja toczy się w roku 1602 roku gdzie wraz z resztą braci bierze udział w polowaniu na czarownice. Jednak niespodziewane spotkanie swojej przyrodniej siostry Anny Held, osądzanej o wykonywanie czarów, powoduje że Laymann odrzuca regułę klasztoru. Podstępnie zakrada się do biblioteki gdzie studiuje zakazane księgi. Zostaje przyłapany przez swojego nauczyciela brata Jakoba i uwięziony w lochu.

Główny bohater spotyka tam starszego człowieka Lugaida Vandroiy, który przedstawia mu się jako druid (Reach Out For The Light). Spotkany człowiek opowiada Gabrielowi o innym zagrożonym wymiarze – Avantasii. W zamian za pomoc w uratowaniu Avantasii druid obiecuje uratowanie Anny. Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy. Vandroiy używa go by przenieść Gabriela do równoległego świata.

W tym czasie biskup Mainz Johann Adam von Bicken, brat Jakob oraz rządca Falk von Kronberg są w drodze do Rzymu gdzie zamierzają spotkać się z papieżem Clemensem VIII (Glory of Rome). Niosą ze sobą także księgę odkrytą przez Gabriela. Wedle starożytnego zapisu wynika że księga jest ostatnią siódmą częścią pieczęci, która w całości daje właścicielowi absolutną wiedzę gdy tylko dostanie się on do wieży wyznaczającej środek Avantasii.

Gdy Gabriel dostaje się do równoległego świata jest powitany przez dwóch jego mieszkańców elfa Elderana oraz krasnoluda Regrina (Inside). Opowiadają mu o toczącej się wojnie przeciwko siłom zła oraz o planach papieża (Sign Of The Cross). Jeżeli papież użyje pieczęci połączenie między Avantasią a światem ludzi zostanie zamknięte, a oba światy dotkną straszliwe kataklizmy. Gabriel przybywa w momencie gdy Clemens VIII rozmawia z tajemniczym głosem dochodzącym z wieży. Zręcznemu Gabrielowi udaje się skraść pieczęć papieżowi po czym zanosi ją do miasta elfów (The Tower). Zdarzenie to kończy pierwszy album.

Jednak Gabriel nie jest usatysfakcjonowany. Chce dowiedzieć się więcej o świecie Avantasii dlatego Elderane wysyła nowicjusza do drzewa poznania. Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice). Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach. Kielich jest więzieniem dla ogromnej ilości torturowanych dusz, artefakt strzeżony jest także przez siejącą postrach bestię. Mimo niepowodzeń delfickich wypraw, Gabriel i Regrin powracają na ziemię by zmierzyć się z bestią. Przyjaciele znajdują kielich i przewracają go co umożliwia ucieczkę duszom. Przebudzona bestia zabija jednak krasnoluda, Gabrielowi udaje się uciec.

Gabriel wraca do Vandroiya, który czekał na niego. Druid spełnia obietnice, zakrada się do więzienia by uwolnić Annę. Jednak znajduje tam „przemienionego” brata Jakoba który także chce uwolnić Annę. Falk von Kronberg nakrywa ich i każe aresztować. Rozpoczęła się walka w której poległ Vandroiy raniony przez Kronberga, który później zostaje uśmiercony przez brata Jakoba. Anna ucieka by ponownie złączyć się z Gabrielem. Podążają wspólnie nieznaną drogą w przyszłość (Into The Unknown).
'''

In [40]:
from langdetect import detect
from random import choice
from nltk.corpus import stopwords
import numpy as np
import re
import string
import networkx as nx
from collections import Counter

from nltk.tokenize import sent_tokenize, word_tokenize
from typing import List
from functools import reduce
import operator
from gensim.models import Word2Vec
from IPython.core.display import display, HTML
from nltk.cluster.util import cosine_distance

In [3]:
STOP_WORDS = set(stopwords.words('english'))
from spacy.lang.pl.stop_words import STOP_WORDS

In [49]:
vector = List[float]

def cos_sim(v: vector, w: vector):
    if ((sum(v) == 0) or (sum(w) == 0)):
        return 0
    else:
        return 1 - cosine_distance(v, w)


In [52]:
def retrieve_stop_words():
    lang = detect(choice(text.split()))
    if lang == 'en':
        STOP_WORDS = set(stopwords.words('english'))
    elif lang == 'pl':
        from spacy.lang.pl.stop_words import STOP_WORDS
    
    return STOP_WORDS

In [53]:
STOP_WORDS = retrieve_stop_words()

In [54]:
class Preprocessing(object):
    def __init__(self, text):
        self.text = text
        self.oryg = text

    def lower(self):
        self.text = self.text.lower() 
        return self.text
    
    def remove_punctuation(self):
        self.text = self.text.translate(self.text.maketrans('', '', string.punctuation.replace('.', '')))
        return self.text 
    
    def remove_stop_words(self):
        self.text = ' '.join([word for word in self.text.split() if word not in STOP_WORDS])
        return self.text
    
    def remove_digits(self):
        self.text = re.sub(r'[\d+]', '', self.text)
        return self.text
    
    def sentence_tokenize(self):
        self.text = sent_tokenize(self.text)
        self.text = [sent.replace('.','') for sent in self.text]
        return self.text
    
    def basic_pipeline(self):
        self.lower()
        self.remove_digits()
        self.remove_punctuation()
        self.remove_stop_words()
        self.sentence_tokenize()
        return self.text

    def __call__(self):
        return self.text

In [55]:
def prepare_text(text):
    cleaned_text = Preprocessing(text)
    sentences = cleaned_text.basic_pipeline()
    raw_sentences = Preprocessing(text).sentence_tokenize()
    return sentences, raw_sentences

In [56]:
sentences, raw_sentences = prepare_text(text)

In [57]:
sentences

['głównym bohaterem gabriel laymann nowicjusz klasztoru dominikanów mainz',
 'akcja toczy wraz resztą braci bierze udział polowaniu czarownice',
 'niespodziewane spotkanie swojej przyrodniej siostry anny held osądzanej wykonywanie czarów powoduje laymann odrzuca regułę klasztoru',
 'podstępnie zakrada biblioteki studiuje zakazane księgi',
 'zostaje przyłapany swojego nauczyciela brata jakoba uwięziony lochu',
 'główny bohater spotyka starszego człowieka lugaida vandroiy przedstawia druid reach out for the light',
 'spotkany człowiek opowiada gabrielowi innym zagrożonym wymiarze – avantasii',
 'zamian pomoc uratowaniu avantasii druid obiecuje uratowanie anny',
 'razem udaje uciec lochów breaking away czym vandroiy zabiera gabriela kamieniołomu ukryty portal łączący oba światy',
 'vandroiy używa przenieść gabriela równoległego świata',
 'czasie biskup mainz johann adam von bicken brat jakob rządca falk von kronberg drodze rzymu zamierzają spotkać papieżem clemensem glory of rome',
 'nios

In [58]:
all_words = [elem.split() for elem in sentences]
all_words_flattened = reduce(operator.concat, all_words)

In [31]:
# def make_representation(tokens):
#     def return_tokens(tokens):
#         return [int(token in tokens) for token in all_words_flattened]
    
#     X = list(map(return_tokens, tokens))

#     return X

In [59]:
from tensorflow.keras.preprocessing import text as kpt
from tensorflow.keras.preprocessing import sequence
def make_representationX(sentences, max_top_words = 100, max_sent_len = 20):
    tokenizer = kpt.Tokenizer(num_words=max_top_words)
    tokenizer.fit_on_texts(sentences)
    seq = tokenizer.texts_to_sequences(sentences)
    return sequence.pad_sequences(seq, maxlen = max_sent_len, padding='post', truncating='post')


In [60]:
sim_matrix = np.zeros((len(sentences), len(sentences)))

for sent1 in range(len(sentences)):
    for sent2 in range(len(sentences)):
        a = make_representationX(sentences)[]

for n, sent1 in enumerate(sentences):
    sent1 = sentences[n]
    for m, sent2 in enumerate(sentences):
        sent2 = sentences[m]
        if sent1 != sent2:
            a = np.divide(make_representationX(sentences)[n], make_representationX(sentences)[n].max())
            b = np.divide(make_representationX(sentences)[m], make_representationX(sentences)[m].max())
            if ((sum(a) !=0) and (sum(b) != 0)):
                sim_matrix[n][m] = cos_sim(a,b)

c:\Users\mariu\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
c:\Users\mariu\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [86]:
for elem in  sim_matrix:
    print(elem)

[0.         0.79631117 0.62869267 0.62260581 0.65174129 0.57337459
 0.92101927 0.75391551 0.48859775 0.31554602 0.62036545 0.45783916
 0.64934738 0.36420498 0.49137591 0.55627909 0.48641874 0.33124846
 0.         0.45783916 0.56019901 0.40839497 0.62588288 0.45783916
 0.45783916 0.45783916 0.52963816 0.45783916 0.33499721 0.18944409
 0.44991977 0.5953729  0.         0.45783916]
[0.79631117 0.         0.7596697  0.75444144 0.78572802 0.68844116
 0.85837102 0.77212587 0.63609804 0.56173904 0.74437746 0.30178203
 0.48019069 0.55337589 0.32382225 0.53046603 0.31973627 0.50783393
 0.         0.30178203 0.3683893  0.57499298 0.4630593  0.30178203
 0.30178203 0.30178203 0.53717844 0.30178203 0.57035213 0.49665329
 0.50910308 0.49003541 0.         0.30178203]
[0.62869267 0.7596697  0.         0.54015643 0.64671994        nan
 0.62956066 0.5531744         nan 0.47289148 0.5139925  0.25646823
 0.40679223 0.46585193 0.27510793 0.44887569 0.27049428 0.42860952
 0.         0.25646823 0.3118714  0.4

In [38]:
scores = nx.pagerank(nx.from_numpy_array(sim_matrix))
scores

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')

In [59]:
summary = sorted(((scores[i], s) for i,s in enumerate(raw_sentences)), reverse = True)

In [74]:
summarized_sents = [sent[1] for sent in summary[:10]]
summarized_sents

['Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice)',
 'Jednak Gabriel nie jest usatysfakcjonowany',
 'Gabriel wraca do Vandroiya, który czekał na niego',
 'Spotkany człowiek opowiada Gabrielowi o innym zagrożonym wymiarze – Avantasii',
 'Zręcznemu Gabrielowi udaje się skraść pieczęć papieżowi po czym zanosi ją do miasta elfów (The Tower)',
 'Niosą ze sobą także księgę odkrytą przez Gabriela',
 'Vandroiy używa go by przenieść Gabriela do równoległego świata',
 'Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach',
 'Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy',
 'Anna ucieka by ponownie złączyć się z Gabrielem']

In [75]:
def display_highlights(raw_sentences, summarized_sents):
    final = []
    for sent in raw_sentences:
        if sent not in summarized_sents:
            final += sent
        else:
            final += f'<span style="background-color:rgba(255,215,0,0.3);"> {sent} </span>'
    display(HTML(''.join([elem for elem in final])))

In [76]:
display_highlights(raw_sentences, summarized_sents)